# Installing packages

In [38]:
import pandas as pd
import pandas as pd
import datetime as dt
import seaborn as sns
import os, glob
import time

# Reading in files

In [39]:
## make one new big file with all csv-files
path = '/Users/petrapinger/neuefische/MPO-Kickstarter/Kickstarter/data'

In [40]:
from pathlib import Path
def read_files():
    """ Only create new csv if not already done"""
    if not Path("./Kickstarter/data/Kickstarter_merged.csv").exists():
        # Read and merge .csv-files
        # Read all .csv-files
        all_files = glob.glob(os.path.join(path, "Kickstarter*.csv"))
        df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
        # Merge .csv-files
        df_merged = pd.concat(df_from_each_file, ignore_index=True)
        df_merged.to_csv('./Kickstarter/data/Kickstarter_merged.csv')
    """Otherwise just read in dataframe from merged .csv file"""
    return pd.read_csv('./Kickstarter/data/Kickstarter_merged.csv', index_col=0)
df = read_files()

In [184]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182264 entries, 1 to 209221
Data columns (total 47 columns):
 #   Column                    Non-Null Count   Dtype          
---  ------                    --------------   -----          
 0   backers_count             182264 non-null  int64          
 1   blurb                     182256 non-null  object         
 2   category                  182264 non-null  object         
 3   converted_pledged_amount  182264 non-null  int64          
 4   country                   182264 non-null  object         
 5   created_at                182264 non-null  int64          
 6   creator                   182264 non-null  object         
 7   currency                  182264 non-null  object         
 8   currency_symbol           182264 non-null  object         
 9   currency_trailing_code    182264 non-null  bool           
 10  current_currency          182264 non-null  object         
 11  deadline                  182264 non-null  int64    

In [108]:
#funtion to extract the year out of the number
def extract_year(number):
    gmtime = time.gmtime(number)
    return gmtime[0]

In [110]:
#funtion to extract the month out of the number
def extract_month(number):
    gmtime = time.gmtime(number)
    return gmtime[1]

In [113]:
#Zeit konvertieren
#adding columns for created_at, launched_at, deadline (only year and month)
df["created_year"] = df.apply(lambda x: extract_year(x["created_at"]), axis=1)  
df["created_month"] = df.apply(lambda x: extract_month(x["created_at"]), axis=1)
df["launched_year"] = df.apply(lambda x: extract_year(x["launched_at"]), axis=1)  
df["launched_month"] = df.apply(lambda x: extract_month(x["launched_at"]), axis=1)
df["deadline_year"] = df.apply(lambda x: extract_year(x["deadline"]), axis=1)  
df["deadline_month"] = df.apply(lambda x: extract_month(x["deadline"]), axis=1)

0          3
1          9
2          6
3          3
4          1
          ..
209217    12
209218     6
209219     9
209220     4
209221     2
Name: deadline_month, Length: 209222, dtype: int64

In [158]:
# function for calculating difference launched_at und deadline and converting to days
def duration(deadline, launched_at):
    duration = deadline - launched_at
    duration_complete = dt.timedelta(seconds=duration)
    return duration_complete.days

In [159]:
# calculating duration column
df["duration_days"] = df.apply(lambda x: duration(x["deadline"], x["launched_at"]), axis=1)

In [155]:
#created_at < launched at < deadline
def preparation(created_at, launched_at):
    duration = launched_at - created_at
    duration_complete = dt.timedelta(seconds=duration) 
    return duration_complete.days

In [155]:
# calculating preparation column
df["preparation"] = df.apply(lambda x: preparation(x["created_at"], x["launched_at"]), axis=1)

In [176]:
# creating dataframe with only the the last of all duplicates entry of the duplicates. 
df_duplicates = df[df.id.duplicated(keep='last')]

In [181]:
# creating dataframe and dropping all duplicates and keep the last ("newest") duplicate 
df = df.drop_duplicates(subset=['id'], keep='last')

In [227]:
#blurb/name/slug abgleichen: Apprently: blurb = description, Name = slug
df[['blurb','name', 'slug']]. head(50)

,blurb,name,slug
1,A colorful Dia de los Muertos themed oracle de...,The Ofrenda Oracle Deck,the-ofrenda-oracle-deck
2,"Electra's long awaited, eclectic Debut Pop/Roc...","Record Electra's Debut Album (Pop, Rock, Class...",record-electras-debut-album-pop-rock-classical
3,The Mist of Tribunal is a turn-based card game...,The Mist of Tribunal - A Card Game,the-mist-of-tribunal-a-card-game
4,"Livng with a brain impairment, what its like t...",Help change the face of Brain Impairment,help-change-the-face-of-brain-impairment
5,Annapolis Chamber Players is a non-for profit ...,Annapolis Chamber Music Project,annapolis-chamber-music-project
6,The Tekalli Duo are commissioning a new piece ...,The Tekalli Duo's New Music and New Concert Se...,the-tekalli-duos-new-music-and-new-concert-season
7,A new slam-bangin' swing album from The Rustmo...,The Swing Album: A Slam-Bangin' Lindy Hoppin' ...,the-swing-album-a-slam-bangin-lindy-hoppin-party
10,Calypso Wings....The Best Wings on Earth!,Calypso Wings: Fast. Fresh. Flavaful,calypso-wings-fast-fresh-flavaful
11,Stories of and by podcasters. How they got st...,Pod Life: Podcaster's Stories,podlife-podcasters-stories
15,What is trust? Delve into the Rose language to...,Trust Workbook: Explore trust issues with a ne...,trust-workbook-explore-trust-issues-with-a-new...


In [199]:
a = "hallo"
len(a)

5

In [220]:
#function to return length of string (number of characters)
def string_len_c(string):
    string_str = str(string)
    string_len = len(string_str)
    return string_len

In [221]:
#function to return length of string (number of words)
def string_len_w(string):
    string_str = str(string)
    string_list = string_str.split()
    string_len = len(string_list)
    return string_len

In [230]:
#function to return length of string (number of words, seperated by "-"
def string_len_slug_w(string):
    string_str = str(string)
    string_list = string_str.split("-")
    string_len = len(string_list)
    return string_len

In [269]:
#function that checks if category or subcategory in slug
def word_in_lists(string, list_cat, list_subcat):
    string_str = str(string)
    string_list = string_str.split("-")
    for word in string_list:
        if word in list_cat:
            return word
        elif word in list_subcat:
            return word
        else:
            return 0

In [222]:
df['blurb_len_c'] = df.apply(lambda x: string_len_c(x['blurb']), axis=1)

In [224]:
df['blurb_len_w'] = df.apply(lambda x: string_len_w(x['blurb']), axis=1)

In [231]:
df['slug_len_w'] = df.apply(lambda x: string_len_slug_w(x['slug']), axis=1)

In [233]:
df['slug_len_c'] = df.apply(lambda x: string_len_c(x['slug']), axis=1)

In [274]:
subcategories = ['Playing Cards', 'Rock', 'Nonfiction', 'Classical Music', 'Music',
       'Restaurants', 'Small Batch', 'Footwear', 'Sculpture',
       'Photography', 'Fashion', 'Mixed Media', 'Software',
       'Performance Art', 'Theater', 'Graphic Design', 'Narrative Film',
       'Television', 'Sound', 'Video Art', 'Printing', 'Anthologies',
       'Art Books', 'Live Games', 'Performances', 'Technology',
       'Civic Design', 'Literary Spaces', 'Taxidermy', 'Vegan', 'Gadgets',
       'Spaces', 'Pet Fashion', 'Poetry', 'Jewelry', 'Puzzles',
       'Camera Equipment', 'Installations', '3D Printing', 'Indie Rock',
       'Events', 'Accessories', 'Shorts', 'Documentary', 'Web',
       'Translations', 'Comics', 'Pottery', 'Tabletop Games', 'Art',
       'Faith', 'Zines', 'Action', 'Public Art', 'Ceramics',
       'Illustration', 'Hip-Hop', 'Cookbooks', 'Conceptual Art',
       'Wearables', 'Food', 'Thrillers', 'Design', 'World Music',
       'Movie Theaters', "Children's Books", 'Academic', 'Video Games',
       'Childrenswear', 'Music Videos', 'Fantasy', 'Product Design',
       'Latin', 'Romance', 'Stationery', 'Typography', 'Comedy',
       'Apparel', 'Animation', 'Fine Art', 'Woodworking', 'Experimental',
       'Digital Art', 'DIY', 'Crafts', 'Gaming Hardware', 'Glass',
       'Painting', 'Architecture', 'Young Adult', 'R&B', 'People',
       'Film & Video', 'Robots', 'Blues', 'Periodicals', 'Kids',
       'Candles', 'Knitting', 'Workshops', 'Jazz', 'Pop', 'Dance',
       'Fiction', 'Family', 'Flight', 'Country & Folk', 'Science Fiction',
       'Interactive Design', 'Plays', 'Calendars', 'Print', 'Weaving',
       'Comic Books', 'Webseries', 'Animals', 'Photobooks', 'Food Trucks',
       'Apps', 'Drinks', 'Graphic Novels', 'Webcomics',
       'Fabrication Tools', 'Textiles', 'Publishing', 'Ready-to-wear',
       'Audio', 'Horror', 'Nature', 'Video', 'Makerspaces',
       "Farmer's Markets", 'Punk', 'Literary Journals', 'Bacon',
       'Community Gardens', 'Letterpress', 'Drama', 'Mobile Games',
       'Hardware', 'Farms', 'Places', 'Space Exploration', 'Journalism',
       'Musical', 'Festivals', 'Radio & Podcasts', 'Couture', 'Metal',
       'Electronic Music', 'DIY Electronics', 'Games', 'Immersive',
       'Residencies', 'Embroidery', 'Photo', 'Quilts', 'Crochet',
       'Chiptune']

In [297]:
#makiing lower case subcategories and split if more than one word
subcategories_lower = []
for cat in subcategories:
    cat_split = cat.split()
    for cat in cat_split:
        subcategories_lower.append(cat.lower())

In [275]:
categories = ['Games', 'Music', 'Publishing', 'Food', 'Fashion', 'Art',
       'Photography', 'Technology', 'Theater', 'Design', 'Film & Video',
       'Crafts', 'Comics', 'Dance', 'Journalism']

In [295]:
#makiing lower case subcategories and split if more than one word
categories_lower = []
for cat in categories:
    cat_split = cat.split(" & ")
    for cat in cat_split:
        categories_lower.append(cat.lower())

In [299]:
df['cat_in_slug'] = df.apply(lambda x: word_in_lists(x['slug'], categories_lower, subcategories_lower), axis=1)

In [300]:
df['cat_in_slug'].unique()

array([0, 'art', 'sound', 'live', 'film', 'world', 'poetry', 'painting',
       '3d', 'gaming', 'young', 'food', 'dance', 'jewelry', 'musical',
       'kids', 'documentary', 'fashion', 'pet', 'community', 'graphic',
       'family', 'product', 'drinks', 'music', 'radio', 'action',
       'mobile', 'games', 'fantasy', 'photo', 'playing', 'theaters',
       'animation', 'rock', 'space', 'diy', 'candles', 'animals', 'fine',
       'digital', 'classical', 'pop', 'comic', 'design', 'software',
       'video', 'crochet', 'jazz', 'glass', 'movie', 'nature', 'science',
       'publishing', 'faith', 'metal', 'theater', 'television', 'public',
       'electronic', 'vegan', 'gardens', 'bacon', 'camera', 'small',
       'webseries', 'horror', 'photography', 'indie', 'audio', 'comedy',
       'equipment', 'folk', 'people', 'punk', 'tabletop', 'country',
       'hardware', 'print', 'robots', 'latin', 'restaurants', 'sculpture',
       'woodworking', 'printing', 'knitting', 'illustration',
       'te

In [237]:
import json

In [240]:
df.shape[0]

182264

In [244]:
df_2 = df.iloc[:,2:3]

In [253]:
df_2 = df_2.reset_index()

In [254]:
df_2.head(10)

,index,category
0,1,"{""id"":273,""name"":""Playing Cards"",""slug"":""games..."
1,2,"{""id"":43,""name"":""Rock"",""slug"":""music/rock"",""po..."
2,3,"{""id"":273,""name"":""Playing Cards"",""slug"":""games..."
3,4,"{""id"":48,""name"":""Nonfiction"",""slug"":""publishin..."
4,5,"{""id"":36,""name"":""Classical Music"",""slug"":""musi..."
5,6,"{""id"":36,""name"":""Classical Music"",""slug"":""musi..."
6,7,"{""id"":14,""name"":""Music"",""slug"":""music"",""positi..."
7,10,"{""id"":312,""name"":""Restaurants"",""slug"":""food/re..."
8,11,"{""id"":48,""name"":""Nonfiction"",""slug"":""publishin..."
9,15,"{""id"":48,""name"":""Nonfiction"",""slug"":""publishin..."


In [255]:
df_2['category_parent_id'] = pd.DataFrame([json.loads(df_2["category"][i]).get("parent_id") for i in range(df_2.shape[0])])
df_2['category_id'] = pd.DataFrame([json.loads(df_2["category"][i]).get("id") for i in range(df_2.shape[0])])
df_2['category_name'] = pd.DataFrame([json.loads(df_2["category"][i]).get("name") for i in range(df_2.shape[0])])
df_2['category_parent_id'] = df_2['category_parent_id'].fillna(0).astype("int")

In [256]:
df_2['category_name'].unique()

array(['Playing Cards', 'Rock', 'Nonfiction', 'Classical Music', 'Music',
       'Restaurants', 'Small Batch', 'Footwear', 'Sculpture',
       'Photography', 'Fashion', 'Mixed Media', 'Software',
       'Performance Art', 'Theater', 'Graphic Design', 'Narrative Film',
       'Television', 'Sound', 'Video Art', 'Printing', 'Anthologies',
       'Art Books', 'Live Games', 'Performances', 'Technology',
       'Civic Design', 'Literary Spaces', 'Taxidermy', 'Vegan', 'Gadgets',
       'Spaces', 'Pet Fashion', 'Poetry', 'Jewelry', 'Puzzles',
       'Camera Equipment', 'Installations', '3D Printing', 'Indie Rock',
       'Events', 'Accessories', 'Shorts', 'Documentary', 'Web',
       'Translations', 'Comics', 'Pottery', 'Tabletop Games', 'Art',
       'Faith', 'Zines', 'Action', 'Public Art', 'Ceramics',
       'Illustration', 'Hip-Hop', 'Cookbooks', 'Conceptual Art',
       'Wearables', 'Food', 'Thrillers', 'Design', 'World Music',
       'Movie Theaters', "Children's Books", 'Academic', 'Vi

In [261]:
parents_dict = {1: "Art", 3: "Comics", 6: "Dance", 7: "Design", 9: "Fashion", 10: "Food",
                11: "Film & Video", 12: "Games", 13: "Journalism", 14: "Music", 15: "Photography", 16: "Technology",
               17: "Theater", 18: "Publishing", 26: "Crafts"}

In [259]:
empty = []
for i in range(df_2.shape[0]):
    if df_2["category_parent_id"][i] != 0:
        empty.append(df_2["category_parent_id"][i])
    else:
        empty.append(df_2["category_id"][i])

In [263]:
df_2["filled_parent"] = pd.DataFrame(empty)

In [264]:
df_2["parent_name"] = df_2["filled_parent"].apply(lambda x: parents_dict.get(x))

In [265]:
df_2["parent_name"].unique()

array(['Games', 'Music', 'Publishing', 'Food', 'Fashion', 'Art',
       'Photography', 'Technology', 'Theater', 'Design', 'Film & Video',
       'Crafts', 'Comics', 'Dance', 'Journalism'], dtype=object)